<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/03.%20Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-retrieval} -->

# Understanding Retrieval Question Answering

### Setup

In [1]:
%pip install -Uqqq rich openai tiktoken wandb langchain unstructured tabulate pdf2image chromadb

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os, random
from pathlib import Path
import tiktoken
from getpass import getpass
from rich.markdown import Markdown
from pprint import pprint

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [2]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


## Langchain

[LangChain](https://docs.langchain.com/docs/) is a framework for developing applications powered by language models. We will use some of its features in the code below. Let's start by configuring W&B tracing. 

In [3]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "llmapps"

## Parsing documents

We will use a small sample of markdown documents in this notebook. Let's find them and make sure we can stuff them into the prompt. That means they may need to be chunked and not exceed some number of tokens. 

In [4]:
MODEL_NAME = "text-davinci-003"
# MODEL_NAME = "gpt-4"

In [5]:
from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.md")
    return dl.load()

documents = find_md_files('../dfinity_md/')
len(documents)

5

In [6]:
# We will need to count tokens in the documents, and for that we need the tokenizer
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

In [7]:
# function to count the number of tokens in each document
def count_tokens(documents):
    token_counts = [len(tokenizer.encode(document.page_content)) for document in documents]
    return token_counts

count_tokens(documents)

[1372, 2423, 248, 1560, 996]

We will use `LangChain` built in `MarkdownTextSplitter` to split the documents into sections. Actually splitting `Markdown` without breaking syntax is not that easy. This splitter strips out syntax.
- We can pass the `chunk_size` param and avoid lenghty chunks.
- The `chunk_overlap` param is useful so you don't cut sentences randomly. This is less necessary with `Markdown`

The `MarkdownTextSplitter` also takes care of removing double line breaks and save us some tokens that way.

In [8]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter = MarkdownTextSplitter(chunk_size=1000)
document_sections = md_text_splitter.split_documents(documents)
len(document_sections), max(count_tokens(document_sections))

(40, 261)

let's look at the first section

In [9]:
print(f"number of document_sections: {len(document_sections)}")

Markdown(document_sections[0].page_content)


number of document_sections: 40


Module 4: Autonomous Governance                                                                                    

In this module, you will build upon your work from modules 2 and 3 by implementing a "governance" canister that    
will act as the decentralized arbiter of our auction platform's feature upgrades.                                  

Background                                                                                                         

Refer to our section on Autonomous Canisters in Module 1 for a quick intro to this topic.                          

We ultimately want to create an automated canister that allows stakeholders to propose changes to our App.mo file. 
The governance canister enables these stakeholders to then vote on open proposals and will automatically migrate   
App.mo and replace it with the new, proposed file. This enables us to create an "autonomous" system by which edits 
are suggested and then voted upon, ensuring that stakeholders will have a guaranteed say in the process if         
initially given that power.                                                                                        

Your Task                                                                                                          

We have provided you with starter code in Governor.mo - your job is to finish implementing all necessary methods   
such that we have a functional governance canister.

## Embeddings

Let's now use embeddings with a vector database retriever to find relevant documents for a query. 

In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)

We can create a retriever from the db now, we can pass the `k` param to get the most relevant sections from the similarity search

In [11]:
retriever = db.as_retriever(search_kwargs=dict(k=3))

In [14]:
# query = "How can I share my  report with my team members in a public W&B project?"
query = "What are the steps required to complete the methods: 'auctionItem' and 'makeBid'"
os.environ["WANDB_NOTEBOOK_NAME"] = "05. Retrieval_dfinity.ipynb"
docs = retriever.get_relevant_documents(query)

In [15]:
# Let's see the results
for doc in docs:
    print(doc.metadata["source"])
    
Markdown(docs[0].page_content)

../dfinity_md/module-2.md
../dfinity_md/module-2.md
../dfinity_md/module-2.md


Main.mo                                                                                                            

main.mo has one function, setup, which calls the constructors of the relevant canisters needed to create the entire
application - namely, we need to establish a Balances canister to store balances, an App canister to control the   
actual process of creating and managing auctions,  and a Governor to control the governance structure for automated
upgrades.                                                                                                          

Specification                                                                                                      

Task: Complete the implementation of the auctionItem and makeBid methods in App.mo.                                

auctionItem creates a new Auction and adds it to the auctions list                                                 

Use the name, description, and url parameters are used to create a new Item (by calling the makeItem helper        
function defined below).                                                                                           

Use this item to create a new auction, using the makeAuction helper, and finally add this auction to our auctions  
hash map using a unique id.

## Stuff Prompt

We'll now take the content of the retrieved documents, stuff them into prompt template along with the query, and pass into an LLM to obtain the answer. 

In [16]:
from langchain.prompts import PromptTemplate

prompt_template = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

context = "\n\n".join([doc.page_content for doc in docs])
prompt = PROMPT.format(context=context, question=query)

Use langchain to call openai chat API with the question

In [17]:
from langchain.llms import OpenAI

llm = OpenAI()
response = llm.predict(prompt)
Markdown(response)

For the 'auctionItem' method, the steps required are to create a new item using the name, description and url      
parameters, create a new auction using the makeAuction helper, and add the auction to the auctions hash map using a
unique id. For the 'makeBid' method, the steps required are to check that the bidder has enough money in their     
balance, retrieve the auction stored at the auctionId, check that the new bid amount is greater than the auction's 
current bid, and update the corresponding Auction accordingly.

## Using Langchain

Langchain gives us tools to do this efficiently in few lines of code. Let's do the same using `RetrievalQA` chain.

In [18]:
from langchain.chains import RetrievalQA

print(query)
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
result = qa.run(query)

Markdown(result)

What are the steps required to complete the methods: 'auctionItem' and 'makeBid'


To complete the 'auctionItem' method, you need to use the name, description, and url parameters to create a new    
Item, create a new Auction using the makeAuction helper, and add this auction to the auctions hash map using a     
unique id. To complete the 'makeBid' method, you need to check that the bidder has enough money in their balance to
make the specified bid, retrieve the auction stored at auctionId, check that the new bid amount is greater than the
auction's current bid, and update the corresponding Auction accordingly. Finally, you need to return #ok() if the  
bid is valid, or one of the corresponding error variants if the bid is invalid.

In [ ]:
import wandb
wandb.finish()